Pipeline โดยรวม

Download audio จาก YouTube (wav)

Normalize format → mono / 22050 Hz

ตัด silence + แยกเป็นประโยค (VAD)

แยกผู้พูด (Speaker Diarization)

ตั้ง label ผู้พูด → filter เฉพาะ speaker ที่ต้องการ

ได้ไฟล์สุดท้าย:
1 ประโยค = 1 wav / speaker เดียว / ไม่มีเพลง

In [2]:
# !pip install yt-dlp torch torchaudio pyannote.audio

In [3]:
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


In [4]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
import os
import subprocess
import shutil

import librosa
import soundfile as sf
import torch

from pyannote.audio import Pipeline


c:\Users\Phonenet\anaconda3\envs\audioenv\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [6]:
def download_youtube_audio(youtube_id, out_dir="yt_audio"):
    os.makedirs(out_dir, exist_ok=True)

    url = f"https://www.youtube.com/watch?v={youtube_id}"

    cmd = [
        "yt-dlp",
        "-f", "bestaudio/best",
        "--extract-audio",
        "--audio-format", "wav",
        "--audio-quality", "0",
        "--ffmpeg-location", "C:\\ffmpeg\\bin",  # 🔧 แก้ให้ตรงเครื่อง
        "--postprocessor-args", "ffmpeg:-ac 1",  # mono only
        "-o", f"{out_dir}/{youtube_id}.%(ext)s",
        url
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(result.stderr)
        raise RuntimeError("yt-dlp failed")

    print(f"✅ downloaded {youtube_id}")


In [7]:
def convert_audio(input_wav, output_wav, sr):
    audio, _ = librosa.load(input_wav, sr=sr, mono=True)
    sf.write(output_wav, audio, sr, subtype="PCM_16")


In [8]:
model, utils = torch.hub.load(
    "snakers4/silero-vad",
    "silero_vad",
    trust_repo=True
)

(get_speech_timestamps, _, read_audio, _, _) = utils


Using cache found in C:\Users\Phonenet/.cache\torch\hub\snakers4_silero-vad_master


In [9]:
def get_sentences_16k(wav_16k_path):
    wav = read_audio(wav_16k_path, sampling_rate=16000)
    timestamps = get_speech_timestamps(
        wav,
        model,
        sampling_rate=16000
    )
    return timestamps


In [ ]:
os.environ["HF_TOKEN"] = ""
    
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=os.environ["HF_TOKEN"]
)


def diarize(wav_16k_path):
    diarization = pipeline(wav_16k_path)
    segments = []

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })

    return segments



c:\Users\Phonenet\anaconda3\envs\audioenv\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
c:\Users\Phonenet\anaconda3\envs\audioenv\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
c:\Users\Phonenet\anaconda3\envs\audioenv\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:53: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend(bac

In [11]:
def summarize_speakers(diar_segments):
    stats = {}

    for seg in diar_segments:
        spk = seg["speaker"]
        dur = seg["end"] - seg["start"]
        stats.setdefault(spk, 0.0)
        stats[spk] += dur

    return stats


In [12]:
def find_speaker_for_sentence(sentence, diar_segments):
    s_start = sentence["start"] / 16000
    s_end = sentence["end"] / 16000

    best_speaker = None
    best_overlap = 0.0

    for seg in diar_segments:
        overlap = max(
            0,
            min(s_end, seg["end"]) - max(s_start, seg["start"])
        )
        if overlap > best_overlap:
            best_overlap = overlap
            best_speaker = seg["speaker"]

    return best_speaker


In [13]:
def export_sentences_22k(
    wav_22k_path,
    sentences,
    diar_segments,
    out_dir,
    target_speaker=None
):
    audio, sr = librosa.load(wav_22k_path, sr=22050, mono=True)
    os.makedirs(out_dir, exist_ok=True)

    idx = 0
    for sent in sentences:
        speaker = find_speaker_for_sentence(sent, diar_segments)

        if target_speaker and speaker != target_speaker:
            continue

        start = int((sent["start"] / 16000) * sr)
        end = int((sent["end"] / 16000) * sr)

        chunk = audio[start:end]
        if len(chunk) < sr * 0.3:  # skip too short
            continue

        fname = f"{speaker}_sent_{idx:04d}.wav"
        sf.write(os.path.join(out_dir, fname), chunk, sr)
        idx += 1


In [14]:
def export_speaker_preview(wav_22k_path, diar_segments, out_dir, max_sec=20):
    audio, sr = librosa.load(wav_22k_path, sr=22050, mono=True)
    os.makedirs(out_dir, exist_ok=True)

    acc = {}
    for seg in diar_segments:
        spk = seg["speaker"]
        if acc.get(spk, 0) >= max_sec:
            continue

        start = int(seg["start"] * sr)
        end = int(seg["end"] * sr)
        chunk = audio[start:end]

        sf.write(
            f"{out_dir}/{spk}_preview.wav",
            chunk,
            sr
        )
        acc[spk] = acc.get(spk, 0) + (seg["end"] - seg["start"])


Phase 1 — วิเคราะห์ speaker (ทำครั้งเดียว)

download

convert → 16k / 22k

diarization

สรุปเวลาพูด

export preview ให้ฟัง

Phase 2 — export dataset (หลังรู้ speaker แล้ว)

load sentence (VAD)

match speaker

export wav 22k

📌 1 file = 1 sentence + speaker ถูกต้อง

In [13]:
youtube_ids = [
    "AxUEkqPH7iQ",
    "zbo7Mk3ryaI",
    "PYkZ1Z2pEc8",
    "VN2VeBgR_3g",
    "RKGQ3MvkFrM"
]

os.makedirs("yt_audio", exist_ok=True)
os.makedirs("work_16k", exist_ok=True)
os.makedirs("data_22k", exist_ok=True)
os.makedirs("speaker_preview", exist_ok=True)

for yt_id in youtube_ids:
    print(f"\n🎬 Analyze {yt_id}")

    raw = f"yt_audio/{yt_id}.wav"
    wav_16k = f"work_16k/{yt_id}.wav"
    wav_22k = f"data_22k/{yt_id}.wav"

    # 1. download
    if not os.path.exists(raw):
        download_youtube_audio(yt_id, "yt_audio")

    # 2. convert
    if not os.path.exists(wav_16k):
        convert_audio(raw, wav_16k, 16000)

    if not os.path.exists(wav_22k):
        convert_audio(raw, wav_22k, 22050)

    # 3. diarization
    diar_segments = diarize(wav_16k)

    # 4. summarize speaker
    stats = summarize_speakers(diar_segments)
    print("Speaker summary:")
    for spk, dur in sorted(stats.items(), key=lambda x: -x[1]):
        print(f"  {spk}: {dur:.1f} sec")

    # 5. export preview (ฟังเพื่อเลือก speaker)
    export_speaker_preview(
        wav_22k_path=wav_22k,
        diar_segments=diar_segments,
        out_dir=f"speaker_preview/{yt_id}"
    )

    print("👉 ฟัง preview แล้วจดว่า speaker ไหนคือคนที่ต้องการ")



🎬 Analyze AxUEkqPH7iQ
✅ downloaded AxUEkqPH7iQ
Speaker summary:
  SPEAKER_01: 1758.4 sec
  SPEAKER_00: 987.0 sec
  SPEAKER_02: 47.8 sec
  SPEAKER_03: 30.1 sec
  SPEAKER_04: 25.6 sec
👉 ฟัง preview แล้วจดว่า speaker ไหนคือคนที่ต้องการ

🎬 Analyze zbo7Mk3ryaI
✅ downloaded zbo7Mk3ryaI
Speaker summary:
  SPEAKER_00: 1822.0 sec
  SPEAKER_01: 648.8 sec
  SPEAKER_02: 96.4 sec
👉 ฟัง preview แล้วจดว่า speaker ไหนคือคนที่ต้องการ

🎬 Analyze PYkZ1Z2pEc8
✅ downloaded PYkZ1Z2pEc8
Speaker summary:
  SPEAKER_03: 3235.8 sec
  SPEAKER_01: 579.0 sec
  SPEAKER_02: 402.9 sec
  SPEAKER_04: 266.5 sec
  SPEAKER_00: 196.9 sec
👉 ฟัง preview แล้วจดว่า speaker ไหนคือคนที่ต้องการ

🎬 Analyze VN2VeBgR_3g
✅ downloaded VN2VeBgR_3g
Speaker summary:
  SPEAKER_01: 1019.7 sec
  SPEAKER_00: 41.6 sec
👉 ฟัง preview แล้วจดว่า speaker ไหนคือคนที่ต้องการ

🎬 Analyze RKGQ3MvkFrM
✅ downloaded RKGQ3MvkFrM
Speaker summary:
  SPEAKER_03: 2971.2 sec
  SPEAKER_01: 1554.6 sec
  SPEAKER_04: 152.1 sec
  SPEAKER_02: 82.2 sec
  SPEAKER_00: 8

In [15]:
TARGET_SPEAKER = {
    "AxUEkqPH7iQ": "SPEAKER_01",
    "zbo7Mk3ryaI": "SPEAKER_00"
}

youtube_ids = TARGET_SPEAKER.keys()

os.makedirs("Lisa", exist_ok=True)

for yt_id in youtube_ids:
    print(f"\n📦 Export dataset {yt_id}")

    wav_16k = f"work_16k/{yt_id}.wav"
    wav_22k = f"data_22k/{yt_id}.wav"

    # 1. sentence from VAD (16k)
    sentences = get_sentences_16k(wav_16k)

    # 2. diarization (reuse)
    diar_segments = diarize(wav_16k)

    # 3. export dataset (22k)
    export_sentences_22k(
        wav_22k_path=wav_22k,
        sentences=sentences,
        diar_segments=diar_segments,
        out_dir=f"dataset/{yt_id}",
        target_speaker=TARGET_SPEAKER[yt_id]
    )

    print(f"✅ Finished dataset {yt_id}")



📦 Export dataset AxUEkqPH7iQ
✅ Finished dataset AxUEkqPH7iQ

📦 Export dataset zbo7Mk3ryaI
✅ Finished dataset zbo7Mk3ryaI
